In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np

In [2]:
train=pd.read_csv('C:/Users/Hp/OneDrive/Documents/IIT KANPUR PYTHON/2nd PROJECT/carvan_train.csv')

In [3]:
test=pd.read_csv('C:/Users/Hp/OneDrive/Documents/IIT KANPUR PYTHON/2nd PROJECT/carvan_test.csv')

# Data cleaning

In [4]:
train.isnull().sum()

V1     0
V2     0
V3     0
V4     0
V5     0
      ..
V82    0
V83    0
V84    0
V85    0
V86    0
Length: 86, dtype: int64

In [5]:
test.isnull().sum()

V1     0
V2     0
V3     0
V4     0
V5     0
      ..
V81    0
V82    0
V83    0
V84    0
V85    0
Length: 85, dtype: int64

In [6]:
for col in ['V1','V5']:
    train[col]=train[col].astype('category',copy=False)
    test[col]=test[col].astype('category',copy=False)

In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5822 entries, 0 to 5821
Data columns (total 86 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   V1      5822 non-null   category
 1   V2      5822 non-null   int64   
 2   V3      5822 non-null   int64   
 3   V4      5822 non-null   int64   
 4   V5      5822 non-null   category
 5   V6      5822 non-null   int64   
 6   V7      5822 non-null   int64   
 7   V8      5822 non-null   int64   
 8   V9      5822 non-null   int64   
 9   V10     5822 non-null   int64   
 10  V11     5822 non-null   int64   
 11  V12     5822 non-null   int64   
 12  V13     5822 non-null   int64   
 13  V14     5822 non-null   int64   
 14  V15     5822 non-null   int64   
 15  V16     5822 non-null   int64   
 16  V17     5822 non-null   int64   
 17  V18     5822 non-null   int64   
 18  V19     5822 non-null   int64   
 19  V20     5822 non-null   int64   
 20  V21     5822 non-null   int64   
 21  V22     5822 n

In [8]:
x=train.drop(['V86'],1)
y=train['V86']

# Model fitting

In [9]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier

In [10]:
params={ 'class_weight':[None,'balanced'], 
        'criterion':['entropy','gini'],
        'max_depth':[None,5,10,15,20,30,50,70],
            'min_samples_leaf':[1,2,5,10,15,20], 
            'min_samples_split':[2,5,10,15,20]
       }

In [11]:
clf=DecisionTreeClassifier()

In [12]:
tree=RandomizedSearchCV(clf,cv=10,
                        param_distributions=params,
                        scoring='roc_auc',
                        n_iter=20,n_jobs=-1,verbose=20)

In [13]:
tree.fit(x,y)

Fitting 10 folds for each of 20 candidates, totalling 200 fits


RandomizedSearchCV(cv=10, estimator=DecisionTreeClassifier(), n_iter=20,
                   n_jobs=-1,
                   param_distributions={'class_weight': [None, 'balanced'],
                                        'criterion': ['entropy', 'gini'],
                                        'max_depth': [None, 5, 10, 15, 20, 30,
                                                      50, 70],
                                        'min_samples_leaf': [1, 2, 5, 10, 15,
                                                             20],
                                        'min_samples_split': [2, 5, 10, 15,
                                                              20]},
                   scoring='roc_auc', verbose=20)

In [14]:
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.5f} (std: {1:.5f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [15]:
report(tree.cv_results_,5)

Model with rank: 1
Mean validation score: 0.73410 (std: 0.03778)
Parameters: {'min_samples_split': 15, 'min_samples_leaf': 5, 'max_depth': 5, 'criterion': 'gini', 'class_weight': None}

Model with rank: 2
Mean validation score: 0.72229 (std: 0.03246)
Parameters: {'min_samples_split': 10, 'min_samples_leaf': 20, 'max_depth': 5, 'criterion': 'entropy', 'class_weight': None}

Model with rank: 3
Mean validation score: 0.71820 (std: 0.03478)
Parameters: {'min_samples_split': 2, 'min_samples_leaf': 5, 'max_depth': 5, 'criterion': 'entropy', 'class_weight': 'balanced'}

Model with rank: 4
Mean validation score: 0.66285 (std: 0.04438)
Parameters: {'min_samples_split': 5, 'min_samples_leaf': 20, 'max_depth': None, 'criterion': 'entropy', 'class_weight': 'balanced'}

Model with rank: 5
Mean validation score: 0.65713 (std: 0.03958)
Parameters: {'min_samples_split': 2, 'min_samples_leaf': 20, 'max_depth': 20, 'criterion': 'entropy', 'class_weight': None}



In [16]:
dtree=DecisionTreeClassifier(**{'min_samples_split': 15, 'min_samples_leaf': 5, 'max_depth': 5, 'criterion': 'gini', 'class_weight': None})

In [17]:
dtree.fit(x,y)

DecisionTreeClassifier(max_depth=5, min_samples_leaf=5, min_samples_split=15)

In [18]:
predictions = dtree.predict(test)

In [20]:
prediction = pd.DataFrame(predictions, columns=['v86'])

In [21]:
prediction.to_csv('sample_submission.csv',index=False)